#Projek Akhir Stechoq Tensorflow
Kelas A Kelompok 4

Projek: Klasifikasi luka bakar dengan CNN berbasis Aplikasi

Nama Anggota Kelompok:
1. Ahmad Yudhistira Fachreza Hidayat
2. Chandra Aditiya
3. Faishal Reyzally Hendriansyah
4. Mohammad Dimas Noufal
5. Rizki Muhammad Ridwan

In [1]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install fastapi uvicorn pyngrok -q

In [3]:
%%writefile main.py
from typing import Union
from fastapi import FastAPI, File, UploadFile, Form
from starlette.responses import HTMLResponse, JSONResponse
import numpy as np
import tensorflow as tf
import cv2
import os
from fastapi.middleware.cors import CORSMiddleware

# Initialize the FastAPI app
app = FastAPI()

# Enable CORS middleware for development purposes
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the model and define the labels
model = tf.keras.models.load_model('/content/nasnetmobile_skin_burn_dropout_classification.h5')
labels = ["1st_degree burn", "2nd_degree burn", "3rd_degree burn", "No Sunburn"]

def classify_image(image_path):
    # Read and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)

    # Predict and return the results
    prediction = model.predict(image)[0]
    predicted_label = np.argmax(prediction)
    result = labels[predicted_label]
    return {'Score': float(prediction[predicted_label]), 'Label': result}

@app.get("/", response_class=HTMLResponse)
async def get_home():
    return """
            <html>
                <head>
                    <title>Welcome to Skin Burn Check</title>
                    <style>
                        body {
                            display: flex;
                            justify-content: center;
                            align-items: center;
                            height: 100vh;
                            margin: 0;
                            font-family: Arial, sans-serif;
                            background-color: #f0f0f0;
                        }
                        .card {
                            background: white;
                            padding: 20px;
                            border-radius: 10px;
                            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                            text-align: center;
                        }
                        h1 {
                            margin-bottom: 20px;
                        }
                        .file-input {
                            display: none;
                        }
                        .custom-file-upload {
                            display: inline-block;
                            padding: 10px 20px;
                            cursor: pointer;
                            border: 2px solid #4CAF50;
                            border-radius: 5px;
                            color: #4CAF50;
                            background-color: white;
                            transition: all 0.3s ease;
                        }
                        .custom-file-upload:hover {
                            background-color: #4CAF50;
                            color: white;
                        }
                        .file-container {
                            margin-bottom: 20px;
                        }
                        button {
                            background-color: #4CAF50;
                            color: white;
                            border: none;
                            padding: 10px 20px;
                            border-radius: 5px;
                            cursor: pointer;
                            transition: all 0.3s ease;
                        }
                        button:hover {
                            background-color: #45a049;
                        }
                        #preview {
                            margin-top: 20px;
                            margin-bottom: 20px;
                            display: none;
                            max-width: 100%;
                            width: 350px;
                            height: 350px;
                            margin-left: auto;
                            margin-right: auto;
                        }
                        .info-section {
                            margin-top: 30px;
                            padding: 20px;
                            background-color: #fff;
                            border-radius: 10px;
                            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                        }
                        .info-section h2, .info-section h3 {
                            color: #333;
                        }
                        .info-section p, .info-section ul {
                            color: #666;
                            line-height: 1.6;
                        }
                        .info-section ul {
                            list-style-type: disc;
                            margin-left: 20px;
                        }
                    </style>
                </head>
                <body>
                    <div class="card">
                        <h1>Welcome to Our Skin Burn Check</h1>
                        <p>Please take a picture to check skin burn level</p>
                        <form id="upload-form" action="/upload" method="post" enctype="multipart/form-data">
                            <div class="file-container">
                                <label for="file-upload" class="custom-file-upload">
                                    Choose Image
                                </label>
                                <input id="file-upload" class="file-input" type="file" name="file" accept="image/*" capture="camera" onchange="previewFile()">
                            </div>
                            <img id="preview" src="" alt="Image preview">
                            <button type="button" onclick="uploadImage()">Upload Image</button>
                        </form>
                        <div id="result" style="margin-top: 20px;"></div>
                        <!-- Elemen baru untuk menampilkan penjelasan -->
                        <div id="explanation" style="margin-top: 20px;"></div>
                    </div>
                    <script>
                        function previewFile() {
                            const preview = document.getElementById('preview');
                            const file = document.getElementById('file-upload').files[0];
                            const reader = new FileReader();

                            reader.addEventListener('load', function () {
                                preview.src = reader.result;
                                preview.style.display = 'block';
                            }, false);

                            if (file) {
                                reader.readAsDataURL(file);
                            }
                        }

                        async function uploadImage() {
                            const form = document.getElementById('upload-form');
                            const formData = new FormData(form);
                            const response = await fetch('/upload', {
                                method: 'POST',
                                body: formData
                            });
                            const result = await response.json();
                            document.getElementById('result').innerHTML = `<strong>Label:${result.Label}, Score: ${parseFloat(result.Score * 100).toFixed(2)}%</strong>`;

                            // Menentukan penjelasan berdasarkan label
                            let explanation = "";
                            switch(result.Label) {
                                case "1st_degree burn":
                                    explanation = "<p>Luka bakar derajat satu, biasa disebabkan oleh sunburn dalam waktu lama, api dari kompor, bahan kimia, dan sebagainya </p>" +
                                                  "<p>Bekas luka akan memudar dalam 7-10 hari dengan adanya pergantian kulit dan sel-sel baru. Pemberian salep akan membantu meredakan nyeri</p>" ;
                                    break;
                                case "2nd_degree burn":
                                    explanation = "<p>Luka bakar derajat dua (fibrinous exudate) merupakan luka bakar yang memengaruh kulit luar (epidermis) dan kulit tengah (dermis) </p>" +
                                                  "<p>Bekas luka akan pulih dalam 1-3 minggu, sangat dianjurkan untuk memberikan obat anitibiotik.</p>";
                                    break;
                                case "3rd_degree burn":
                                    explanation = "<p>Luka bakar derajat tiga meruapakn luka bakar yang merusak hingga lapisan kulit terbawah dan memicu kerusakan pada tulang dan otot</p>" +
                                                  "<p>Pasien dengan luka bakar derajat tiga memerlukan prosedur operasi, termasuk suntikan antibiotik dan infus.</p>";
                                    break;
                                case "No Sunburn":
                                    explanation = "<p>Luka bakar tidak terdeteksi, gunakan sunscreen dan pakaian tertutup untuk melindungi kulit dari panas.</p>";
                                    break;
                                default:
                                    explanation = "Please consult a medical professional for a detailed examination.";
                                    break;
                            }

                            // Menampilkan penjelasan
                            document.getElementById('explanation').innerHTML = explanation;
                        }
                    </script>
                </body>
            </html>
    """

@app.post("/upload")
async def upload_image(file: UploadFile = File(...)):
    # Save the uploaded file
    os.makedirs('temp', exist_ok=True)
    file_location = f"temp/{file.filename}"
    with open(file_location, "wb+") as file_object:
        file_object.write(file.file.read())

    # Classify the image
    result = classify_image(file_location)

    # Remove the file after processing
    os.remove(file_location)

    return JSONResponse(content=result)


Writing main.py


In [ ]:
from pyngrok import ngrok
import uvicorn

# Setup ngrok
ngrok.set_auth_token('2h5RXMbRCydqW5eEXDJejqrlaEf_49NQvU2gYGAXxN7EbXvpL')

public_url = ngrok.connect(addr="8000", proto="http", bind_tls=True)
print(f"Public URL: {public_url}")

# Start server
uvicorn.run(app='main:app', host="0.0.0.0", port=8000, reload=True, use_colors=True)

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [163] using WatchFiles


Public URL: NgrokTunnel: "https://c1c6-34-170-231-63.ngrok-free.app" -> "http://localhost:8000"
